In [2]:
import re
import pandas as pd
import string
import pickle
from textblob import TextBlob
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from nltk.corpus import stopwords
from utils import make_stopwords

<ipython-input-2-83bbd76b64dc>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


## Function for importing books, cleaning text

In [2]:
def import_book(num):
    """
    Given number of book (1-14), return list of chapter strings for a book
    """
    with open('books/book{}.txt'.format(num), 'r') as book:
        text = book.read()
    raw_chapters = re.split('CHAPTER|EPILOGUE',text)
    chapters = [' '.join(chapter.split()) for chapter in raw_chapters]
    chapters = [chapter.lower() for chapter in chapters]
    return chapters

def clean_chapter(text, chapter_name):
    """
    Given text of a chapter, return cleaned string
    """
    
    #Get rid of whitespace, punctuation, numbers, words prologue and epilogue
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…▼—]', '', text)
    text = re.sub('prologue|epilogue', '', text)
    
    #Get rid of the chapter name from the beginning of the chapter, if it's there
    beginning = text[:100]
    end = text[100:]
    beginning = re.sub(chapter_name.lower().strip(), '',beginning)
    
    text= (beginning+end).strip()
    text=' '.join(text.split())
    return text

def clean_titles(text):
    """
    Clean chapter titles
    """
    text = ''.join(i for i in text if not i.isdigit())
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('Epilogue', '', text)
    text = re.sub('Prologue', '', text)
    return text

plural_exceptions = ['aes', 'lews', 'talmanes', 'pips', 'elyas']
def singularize_chapter(text, exceptions = plural_exceptions):
    """
    Given chapter text, return string with no plurals/posessives. Leave out anything in plural exceptions
    This doesn't do much for topic modeling and it takes a while, so if you're working with a larger corpus
    it may be better to leave this out. I did it mostly for aesthetic reasons
    """
    tb = TextBlob(text)
    words = tb.words
    sing_list=[word if word in exceptions else word.singularize() for word in words]
    return ' '.join(sing_list)



In [3]:
sw=make_stopwords()

## Import and clean books 

In [4]:
all_text = []
for num in range(1,15):
    book = import_book(num)
    if num == 4:
        book = book[1:]
    print('Book', num, len(book), "chapters")
    all_text.extend(book)
    

Book 1 54 chapters
Book 2 51 chapters
Book 3 57 chapters
Book 4 58 chapters
Book 5 57 chapters
Book 6 57 chapters
Book 7 42 chapters
Book 8 32 chapters
Book 9 36 chapters
Book 10 32 chapters
Book 11 39 chapters
Book 12 52 chapters
Book 13 59 chapters
Book 14 51 chapters


In [5]:
text_df = pd.DataFrame(all_text, columns = ['text'])
text_df

,text
0,prologue dragonmount the palace still shook oc...
1,"1 an empty road the wheel of time turns, and a..."
2,2 strangers when rand and mat carried the firs...
3,3 the peddler clusters of pots clattered and b...
4,4 the gleeman the door of the inn banged shut ...
...,...
672,46 to awaken rand broke free from the darkness...
673,"47 she fought it off, but barely. the forsaken..."
674,48 so many dead. hundreds of thousands of men ...
675,49 beneath that was only emptiness. in the wol...


In [8]:
chapter_names = pd.read_csv('data/chapters.csv', usecols = ['chapter_title', 'book_title', 'chapter_nr'])

In [9]:
chapter_names['chapter_title'] = chapter_names['chapter_title'].apply(lambda s: clean_titles(s))
chapter_names['cumulative_chapter_number'] = chapter_names.index +1 
chapter_names

,chapter_title,chapter_nr,book_title,cumulative_chapter_number
0,Dragonmount,Prologue,The Eye of the World,1
1,An Empty Road,1,The Eye of the World,2
2,Strangers,2,The Eye of the World,3
3,The Peddler,3,The Eye of the World,4
4,The Gleeman,4,The Eye of the World,5
...,...,...,...,...
672,To Awaken,46,A Memory of Light,673
673,Watching the Flow Writhe,47,A Memory of Light,674
674,A Brilliant Lance,48,A Memory of Light,675
675,Light and Shadow,49,A Memory of Light,676


In [10]:
allbooks = pd.concat([chapter_names, text_df], axis=1)
allbooks['text'] = allbooks.apply(lambda x: clean_chapter(x['text'], x['chapter_title']), axis=1)
allbooks

,chapter_title,chapter_nr,book_title,cumulative_chapter_number,text
0,Dragonmount,Prologue,The Eye of the World,1,the palace still shook occasionally as the ear...
1,An Empty Road,1,The Eye of the World,2,the wheel of time turns and ages come and pass...
2,Strangers,2,The Eye of the World,3,when rand and mat carried the first barrels th...
3,The Peddler,3,The Eye of the World,4,clusters of pots clattered and banged as s wag...
4,The Gleeman,4,The Eye of the World,5,the door of the inn banged shut behind the whi...
...,...,...,...,...,...
672,To Awaken,46,A Memory of Light,673,rand broke free from the darkness and entered ...
673,Watching the Flow Writhe,47,A Memory of Light,674,she fought it off but barely the forsaken lean...
674,A Brilliant Lance,48,A Memory of Light,675,so many dead hundreds of thousands of men and ...
675,Light and Shadow,49,A Memory of Light,676,beneath that was only emptiness in the wolf dr...


This is all we really need, but I'm going to 

In [11]:
allbooks.to_pickle('data/allbooks.pkl')

This is all we really need, but I'm going to remove stop words and singularize so that's all done consistently. Tfidf vectorizer can remove stopwords too, but it doesn't hurt to do it here. 

No Stop words

In [12]:
allbooks_nosw = allbooks.copy()
allbooks_nosw['text'] = allbooks_nosw.progress_apply(lambda x: remove_stopwords(x['text']), axis=1)

  0%|          | 0/677 [00:00<?, ?it/s]

In [13]:
allbooks_nosw.to_pickle('data/allbooks_nosw.pkl')

No stop words, signularized

In [14]:
allbooks_sing_nosw = allbooks_nosw.copy()
allbooks_sing_nosw['text'] = allbooks_sing_nosw.progress_apply(lambda x: singularize_chapter(x['text']), axis=1)

  0%|          | 0/677 [00:00<?, ?it/s]

In [15]:
allbooks_sing_nosw.to_pickle('data/allbooks_sing_nosw.pkl')

In [16]:
allbooks_sing_nosw

,chapter_title,chapter_nr,book_title,cumulative_chapter_number,text
0,Dragonmount,Prologue,The Eye of the World,1,palace still shook occasionally earth rumbled ...
1,An Empty Road,1,The Eye of the World,2,wheel time turn age come pas leaving memory be...
2,Strangers,2,The Eye of the World,3,rand mat carried first barrel common room mast...
3,The Peddler,3,The Eye of the World,4,cluster pot clattered banged wagon rumbled hea...
4,The Gleeman,4,The Eye of the World,5,door inn banged shut behind whitehaired spun a...
...,...,...,...,...,...
672,To Awaken,46,A Memory of Light,673,rand broke free darknes entered pattern fully ...
673,Watching the Flow Writhe,47,A Memory of Light,674,fought barely forsaken leaned rock ledge short...
674,A Brilliant Lance,48,A Memory of Light,675,many dead hundred thousand man trolloc lying p...
675,Light and Shadow,49,A Memory of Light,676,beneath emptines wolf dream consumed perrin co...


## Extracting list of characters for network graphing

In [264]:
characters = pd.read_html('https://wot.fandom.com/wiki/Statistical_analysis', match='917,726')[0]

In [265]:
characters = characters.sort_values('Word Count', ascending = False)

In [266]:
characters

,Character,# of POVs,"% of Total POVs (1,379)",Word Count,"% of Word Count (4,373,157)",Average Word Count per POV
6,Rand al'Thor,236,17.1139%,917726,20.9854%,3888.67
7,Perrin Aybara,154,11.1675%,540762,12.3655%,3511.44
5,Egwene al'Vere,130,9.4271%,529812,12.1151%,4075.48
19,Matrim Cauthon,116,8.4119%,486811,11.1318%,4196.65
24,Elayne Trakand,83,6.0189%,355856,8.1373%,4287.42
...,...,...,...,...,...,...
86,Abaldar Yulan,1,0.0725%,117,0.0027%,117.00
57,Luan Norwelyn,1,0.0725%,109,0.0025%,109.00
56,Dyelin Taravin,1,0.0725%,102,0.0023%,102.00
58,Ellorien Traemane,1,0.0725%,83,0.0019%,83.00


In [284]:
char_list = list(characters['Character'])

First names only - good enough for most characters but it does leave out some characters, as well as missing on the fact that many characters have different names, i.e. some call Rand al'Thor or Lews Therin. In a future iteration I'd want to capture this behavior

In [285]:
char_list = [char.split()[0].lower().replace("\'",'') for char in char_list]

Now we need to replace some names with names that actually get used (ie Matrim -> Mat). Also we want to get rid of some characters that we dont want to keep track of in the network (seanchan rider, narrator, etc)

In [287]:
for char in ['matrim', 'galadedrid', 'thomdril', 'lews', 'narrator', 'mazrim', 'quote', 'geofram', 'davram',
            'rodel', 'seanchan']:
    char_list.remove(char)

In [288]:
for char in ['mat', 'galad', 'thom', 'taim', 'bornhald', 'bashere', 'ituralde']:
    char_list.append(char)

In [289]:
char_list=sorted(char_list)

In [290]:
with open('char_list.pkl', 'wb') as f:
    pickle.dump(char_list, f)